In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

# Sample the number of people/vehicles by county segment and year

see  `scripts/sample_counties.py` to run 

In [2]:


#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

### TODO: revisit taking out mobile home owners
df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



In [3]:
counties = pd.read_parquet(datadir+'/population_counties_dataset.parquet', engine='pyarrow')
df = pd.merge(df, counties, on='person_id', how='left')

In [4]:
df.columns

Index(['activity_id', 'person_id', 'mode', 'travel_purpose',
       'previous_activity_type', 'start_time', 'end_time', 'distance_miles',
       'vehicle_type', 'origin_bgrp', 'origin_bgrp_lat', 'origin_bgrp_lng',
       'destination_bgrp', 'destination_bgrp_lat', 'destination_bgrp_lng',
       'origin_land_use_l1', 'origin_land_use_l2', 'origin_building_use_l1',
       'origin_building_use_l2', 'destination_land_use_l1',
       'destination_land_use_l2', 'destination_building_use_l1',
       'destination_building_use_l2', 'origin_lat', 'origin_lng',
       'destination_lat', 'destination_lng', 'weekday', 'household_id',
       'BLOCKGROUP', 'BLOCKGROUP_work', 'BLOCKGROUP_school', 'TRACT',
       'TRACT_work', 'TRACT_school', 'age_group', 'age', 'sex', 'race',
       'ethnicity', 'individual_income_group', 'individual_income',
       'employment', 'education', 'school_grade_attending', 'industry',
       'household_role', 'subfamily_number', 'subfamily_relationship',
       'commute_mo

### Stock rollover model from Gerard of Evs by county, year, vehicle segment, engine (EV, PHEV) and housing (sfh, mfh).

`ev_df`

In [5]:
stock_rollover = pd.read_csv(datadir+'ldv_population_output_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [6]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

Personal Sedan EV 0.25
Personal Sedan PHEV 80.0
Personal Crossover EV 0.3
Personal Crossover PHEV 80.0
Personal Truck/SUV EV 0.49
Personal Truck/SUV PHEV 80.0
Commercial Sedan EV 0.25
Commercial Sedan PHEV 80.0
Commercial Crossover EV 0.3
Commercial Crossover PHEV 80.0
Commercial Truck/SUV EV 0.49
Commercial Truck/SUV PHEV 80.0


# extract the population ldv data from the stock rollover model 
 - What to do about housing == 'other'? 

### include both EV and PHEV

In [7]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV']).reset_index(drop=True)
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
nev_df.drop(columns=['Unnamed: 0'], inplace=True)


See the change in stock over the years

We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes for all the years and segments

In [8]:
nev_df

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,Adams,Commercial Crossover,mfh,EV,0,0,0,0,0,0,0,1,1,1,1,1,1,1
3,Adams,Commercial Crossover,mfh,PHEV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Adams,Commercial Crossover,sfh,EV,0,0,1,1,1,2,2,2,3,3,3,5,5,5
7,Adams,Commercial Crossover,sfh,PHEV,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,Adams,Commercial Sedan,mfh,EV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,Yakima,Personal Sedan,sfh,PHEV,98,2433,4815,6453,7994,9117,10046,10703,11606,12252,12842,13552,14206,14859
1840,Yakima,Personal Truck/SUV,mfh,EV,34,1780,3504,5143,6768,8393,9963,11285,12851,14524,16279,18064,19987,22314
1843,Yakima,Personal Truck/SUV,mfh,PHEV,18,1063,1757,2265,2617,2945,3170,3367,3593,3845,4102,4380,4667,5020
1844,Yakima,Personal Truck/SUV,sfh,EV,133,6521,12813,18796,24713,30637,36335,41141,46932,53211,59811,66703,74132,83107


In [9]:
# get the unique people in the dataframe
pop_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'home_cty', 'building_type']]

In [10]:
# check outputs
df_2035 = pd.read_parquet(os.path.join(datadir, 'county_samples/county_sample_2035.parquet'))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/county_samples/county_sample_2035.parquet'

In [14]:
len(df_2035)

30547656

In [3]:
df_2034 = pd.read_parquet(os.path.join(datadir, 'county_samples/county_sample_2034.parquet'))

# this is the most recent model that should be working on full dataset.

In [15]:
# slice the unique dataframe to only include people in that county
county = 'Yakima'
county_str = county + ' County, WA'
county_cond = pop_df['home_cty'] == county_str 
county_pop_df = pop_df[county_cond].copy()  
county_trip_df = df[df['home_cty'] == county_str].copy() 
nvehicles_sub = nev_df[nev_df['County'] == county].copy()
df.shape, county_trip_df.shape, pop_df.shape, county_pop_df.shape

((25405990, 71), (930192, 71), (5046603, 3), (148278, 3))

In [22]:
total_vehicles_in_yakima = nvehicles_sub['2035'].sum()
total_vehicles_in_yakima, county_pop_df.shape[0], county_pop_df.shape[0]/total_vehicles_in_yakima

(184420, 148278, 0.8040234247912373)

In [19]:
def exclusionary_sampler(df: pd.DataFrame, population_df: pd.DataFrame, nev_df: pd.DataFrame, county: str, year: str) -> pd.DataFrame:
    year = str(year)
    
    # subset the nev_df to only include the county
    # do we prioritize the the most number of vehicles
    # this option has the most remainders
    # nvehicles_sub = nev_df.sort_values(by=year, ascending=False)
    
    # or by ~least number of vehicles
    nvehicles_sub = nev_df

    # Create a list to keep track of selected individuals for each combination
    already_sampled_people = []

    # Create a list to store the cnty dataframes.
    cnty_df_list = []

    

    # Iterate over the county DataFrame and sample individuals from the population DataFrame
    for _, row in nvehicles_sub.iterrows():
        county = row['County']
        vehicle_type = row['Vehicle_type']
        domicile = row['domicile']
        count = row[year]
        # engine = row['Powertrain']
        powertrain = row['Powertrain']

        if count < 0:
            count = 0

        # slice the datafrane to only include people with the correct domicile
        if domicile == 'sfh':
            domicile_cond = population_df['building_type'] == 'single_family'
        else:
            domicile_cond = population_df['building_type'] != 'single_family'

        # filter the county population based on domicile
        filtered_population = population_df[(domicile_cond)]
        print(
            f'Total # of people in county and domicile = {domicile}: {filtered_population.shape[0]}')

        # exclude already selected individuals for this combination
        already_sampled_cond = filtered_population['person_id'].isin(
            already_sampled_people)
        filtered_people = filtered_population[~already_sampled_cond].copy()

        # check if there are more vehicles than people left in the county
        # if so draw from the general population
        if filtered_people.shape[0] <= count:
            
            tot_people_in_sub = filtered_people.shape[0]
            remainder = count - tot_people_in_sub
            # in this case the sampled pop is the entire county leftover.
            sampled_population = filtered_people
            print("total people in county and domicile", tot_people_in_sub)
            print("remainder", remainder)
            print(vehicle_type, domicile, powertrain)
            
            # filtered population is only a cut on the domicile
            # some people with the same domicile will have
            # multiple vehicles.
            sampled_from_domicile_population = population_df.sample(
                n=remainder, replace=False, random_state=42)
            full_county_plus_sampled_poeple = pd.concat([sampled_population, sampled_from_domicile_population])
            sampled_individuals = full_county_plus_sampled_poeple['person_id'].to_list()    
            
            # filtered_people = population_df.copy()
        else:
            # sample 'count' number of individuals
            sampled_population = filtered_people.sample(
                n=count, replace=False, random_state=42)
            sampled_individuals = sampled_population['person_id'].to_list()

        # Update the selected individuals list
        already_sampled_people.extend(sampled_individuals)

        print(f'Sampled {count} individuals from County: {county}, Vehicle Type: {vehicle_type}, Domicile: {domicile}, Powertrain: {powertrain} \n')

        # grab only those selected people with this combination
        # of county, domicile, and vehicle type and powertrain.
        cnty_df = df[(df['person_id'].isin(sampled_individuals))].copy()
        cnty_df['engine'] = powertrain
        cnty_df['segment'] = vehicle_type
        cnty_df['efficiency'] = simdu.segment_efficiency(vehicle_type)
        cnty_df['year'] = str(year)
        ctny_df = simdu.phev_efficiency_milage(cnty_df, powertrain)
        ctny_df['charge_type'] = ctny_df.apply(simdu.map_charge_type, axis=1)
        cnty_df_list.append(cnty_df)

    full_county_df = pd.concat(cnty_df_list)
    return full_county_df

In [13]:
full_county_df = exclusionary_sampler(county_trip_df, county_pop_df, nvehicles_sub, county, 2035)

Total # of people in county and domicile = mfh: 24001
Sampled 1 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 24001
Sampled 0 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: PHEV 

Total # of people in county and domicile = sfh: 124277
Sampled 13 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 124277
Sampled 1 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 24001
Sampled 36 individuals from County: Yakima, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 24001
Sampled 29 individuals from County: Yakima, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: PHEV 

Total #

In [14]:
full_county_df.drop_duplicates(subset=['person_id']).shape, county_pop_df.shape, full_county_df.shape

((148278, 75), (148278, 3), (1150609, 75))

In [26]:
# slice the unique dataframe to only include people in that county
county_list = nev_df['County'].unique()
year = str(2022)

all_county_in_year_list = []
for county in tqdm(county_list):
    county_str = county + ' County, WA'
    county_cond = pop_df['home_cty'] == county_str 
    county_pop_df = pop_df[county_cond].copy()  
    county_trip_df = df[df['home_cty'] == county_str].copy() 
    nvehicles_sub = nev_df[nev_df['County'] == county].copy()
    # df.shape, county_trip_df.shape, pop_df.shape, county_pop_df.shape

    full_county_df = exclusionary_sampler(county_trip_df, county_pop_df, nvehicles_sub, county, year)
    all_county_in_year_list.append(full_county_df)

all_county_in_year_df = pd.concat(all_county_in_year_list)


  0%|          | 0/39 [00:00<?, ?it/s]

Total # of people in county and domicile = mfh: 1386
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 1386
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: PHEV 

Total # of people in county and domicile = sfh: 8166
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 8166
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 1386
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 1386
Sampled 0 individuals from County: Adams, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: PHEV 

Total # of people in cou

  5%|▌         | 2/39 [00:00<00:10,  3.48it/s]

Total # of people in county and domicile = mfh: 1918
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 1918
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: PHEV 

Total # of people in county and domicile = sfh: 12748
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 12748
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 1918
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 1918
Sampled 0 individuals from County: Asotin, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: PHEV 

Total # of peopl

  8%|▊         | 3/39 [00:02<00:37,  1.05s/it]

...

 95%|█████████▍| 37/39 [01:10<00:03,  1.91s/it]

Total # of people in county and domicile = sfh: 115857
Sampled 176 individuals from County: Whatcom, Vehicle Type: Personal Truck/SUV, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 17740
Sampled 0 individuals from County: Whitman, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 17740
Sampled 0 individuals from County: Whitman, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: PHEV 

Total # of people in county and domicile = sfh: 16754
Sampled 0 individuals from County: Whitman, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 16754
Sampled 0 individuals from County: Whitman, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 17740
Sampled 2 individuals from County: Whitman, Vehicle Type: Commercial Sedan, Domicile: mfh, Powertrain: EV 

T

 97%|█████████▋| 38/39 [01:10<00:01,  1.49s/it]

Total # of people in county and domicile = sfh: 16754
Sampled 15 individuals from County: Whitman, Vehicle Type: Personal Truck/SUV, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 16754
Sampled 7 individuals from County: Whitman, Vehicle Type: Personal Truck/SUV, Domicile: sfh, Powertrain: PHEV 

Total # of people in county and domicile = mfh: 24001
Sampled 0 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: EV 

Total # of people in county and domicile = mfh: 24001
Sampled 0 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: mfh, Powertrain: PHEV 

Total # of people in county and domicile = sfh: 124277
Sampled 0 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: EV 

Total # of people in county and domicile = sfh: 124277
Sampled 0 individuals from County: Yakima, Vehicle Type: Commercial Crossover, Domicile: sfh, Powertrain: PHEV 

Tot

100%|██████████| 39/39 [01:13<00:00,  1.88s/it]
